In [1]:
from algopy import graph, graphmat
from algopy.graphmat import GraphMat
from algopy.graph import Graph


# Graphes - Implémentations et parcours

On va voir deux façons d'aborder l'implémentation des graphes: statiquement en utilisant la représentation matricielle ou dynamiquement (ou statico-dynamiquement) à l'aide des listes d'adjacences.

## Exercice 1.1 : Implémentation matricielle

Il s'agit de construire une matrice carrée de taille $|S|$ où $S$ est l'ensemble des sommets du graphes. En ligne $i$ et colonne $j$ on trouve $1$ si le graphe contient l'arcs $(i, j)$ et $0$ sinon. Dans le cas d'un graphe non-orienté la matrice obtenue est symétrique car si l'arcs $(i, j)$ appartient au graphe il en va de même de l'arcs $(i, j)$. 

Voici une manière d'implémenter la représentation matricielle d'un graphe sous Python. 

### Comment est-ce qu'on représente les multigraphes? 

Il suffit de stocker le nombre d'arêtes/arcs entre les sommets $i$ et $j$ dans le coefficient en place $(i, j)$ (et $(j, i)$ dans le cas non-orienté) de notre matrice d'adjacence. 

### Que faire si l'on veut ajouter des coûts sur les arêtes/arcs?

On peut ajouter une matrice de coûts de même taille que la matrice d'adjacence. 

### Quelle complexité en espace pour cette représentation?

La complexité en espace est $O(|S|^2)$

### Quels avantages et incovenients de travailler avec une telle représentation?

Ces avantages et inconvenients ont sens quand ils sont comparés avec le cas de la représentation par listes d'adjacences.

#### Avantages

- Il est très facile de déterminer s'il y a un lien entre deux sommets (pourquoi?). 

- Elle est pratique quand on veut travailler simultanément sur tous les arcs/arêtes d'un graphe. C'est en particulier pour cette raison qu'elle est utilisée pour les algorithmes de recherches des plus courts chemins entre deux sommets quelconques d'un graphe. 

- On a facilement la liste des prédécesseurs d'un sommets : il suffit de parcourir la colonne qui correspond à celui-ci. 

#### Inconvénients

- En comparaison avec la représentation par listes d'adjacences la complexité est un inconvenient dans le cas des graphes non denses ; c'est-à-dire quand le nombre des arcs/arêtes est très petit par rapport à celui des sommets au carré.

- Pour parcourir la liste des successeurs à un sommet on doit parcourir toute la ligne de celui-ci. C'est-à-dire une liste de la taille le nombre de sommets du graphe. Quand on n'a qu'un seul successeur, c'est cher payé. Le fait qu'un tel parcours est le point de départ des parcours en largeur et en profondeur d'un graphe, cette représentation est souvent abandonnée au profit de celle par listes d'adjacences.

## Exercices 1.2 : Implémentation par listes d'adjacences

Dans cette représentation un graphe est représenté par une liste de listes. Une liste du premier niveau correspond à un sommet $s$, les éléments de celle-ci représentent les sommets successeurs de $s</tt>. Dans une représentation totalement dynamique les éléments d'une liste représentant un sommet sont des pointeurs. 

En voici une l'implémentation avec laquelle on travaillera sous Python.

### Comment est-ce qu'on représente les multigraphes? 

Étant donné un sommet $s$ source de plusieurs liens vers un sommet $v$, il suffit de répéter $v$ autant de fois que nécessaire dans la liste d'adjacence de $s$.

### Que faire si l'on veut ajouter des coûts sur les arêtes/arcs?

On peut remplacer les sommets (ou les pointeurs vers ceux-ci dans les listes d'adjacences) par des couples (sommet but, coût de l'arc source -> but).

### Quelle complexité en espace pour cette représentation?

La complexité en espace est $O(|S| + |A|)$.

### Quels avantages et incovenients de travailler avec une telle représentation?

#### Avantages

- Il est très facile de déterminer la liste des successeurs d'un sommet. 

- Complexité bien inférieure au cas de la représentation matricielle, notamment dans le cas des graphes éparses.

#### Inconvénients

- Rechercher si a lien existe entre deux sommets n'est pas facile (pourquoi?). 

- Il faut faire un petit travail pour rechercher la liste des prédecesseurs d'un sommet.

## Exercice 1.3 : Load

### Cas de l'implémentation matricielle

On commence par écrire une fonction qui ajoute un arc/arête entre deux sommets.

In [41]:
def addedge_m(G, src, dst):
    if (src >= G.order or src < 0) or (dst >= G.order or dst < 0): 
        raise IndexError
    if not G.adj[src][dst]:
        G.adj[src][dst] += 1
        if not G.directed and dst != src:
            G.adj[dst][src] += 1

In [42]:
def loadGRA_m(filename):
    f = open(filename)
    digraph = bool(int(f.readline()))
    order = int(f.readline())
    G = GraphMat(order, digraph)
    for line in f.readlines():
        edge = line.strip().split(' ')
        (i, j) = (int(edge[0]), int(edge[1]))
        addedge_m(G, i, j)
    f.close()
    return G

In [44]:
graph_1_m = loadGRA_m("files/digraph1.gra")

In [45]:
graph_2_m = loadGRA_m("files/graph2.gra")

### Cas de l'implémentation par listes d'adjacences

On commence par écrire une fonction qui ajoute un arc au graphe représenté par listes d'adjacences.

In [46]:
def addedge(G, src, dst):
    if (src >= G.order or src < 0) or (dst >=G.order or dst <0): 
        raise IndexError
    if dst not in G.adjlists[src]:
        G.adjlists[src].append(dst)
        if not G.directed and dst != src:
            G.adjlists[dst].append(src)

In [47]:
def loadGRA(filename):
    f = open(filename)
    digraph = bool(int(f.readline()))
    order = int(f.readline())
    G = Graph(order, digraph)
    for line in f.readlines():
        edge = line.strip().split(' ')
        (i, j) = (int(edge[0]), int(edge[1]))
        addedge(G, i, j)
    f.close()
    return G

In [48]:
graph_1 = loadGRA("files/digraph1.gra")

In [49]:
graph_2 = loadGRA("files/graph2.gra")

## Exercice 1.4 : degrés

Chaque sommet d'un graphe a un degré intérieur et extérieur. Le degré intérieur correspond au nombre d'arêtes qui entrent vers ce somment, le degré extérieur au nombre d'arêtes sortantes. 

### Case de l'implémentation par listes d'adjacences

In [50]:
def degrees(G):
    """
    Args:
        G (Graph): digraph
    Returns:
        (din, dout) (list[int], list[int): indegrees and outdegrees of each vertex

    """
    din = [0] * G.order
    dout = [0] * G.order
    for s in range(G.order):
        dout[s] = len(G.adjlists[s])
        for adj in G.adjlists[s]:
            din[adj] += 1
    return (din, dout)

###  Cas de l'implémentation par matrice d'adjacence

In [51]:
def graphdegrees(G):
    """
    Args:
        G (GraphMat): digraph
    Returns:
        (din, dout) (int, int): indegree and outdegree of G

    """
    din = 0
    dout = 0
    for x in range(G.order):
        (din_x, dout_x) = (0, 0)
        for y in range(G.order):
            dout_x += G.adj[x][y]
            din_x += G.adj[y][x]
        din = max(din, din_x)
        dout = max(dout, dout_x)
    return (din, dout)

## Exercice 1.5: dot

### Cas de l'implémentation matricielle

In [52]:
def todot(G):
    (dot, link) = ("digraph ", " -> ") if G.directed else ("graph ", " -- ")
    dot += "G {\n"
    #vertex traversal
    for s in range(G.order):
        # adjacent list traversal
        for v in range(G.order):
            if G.adj[s][v]:             
                if G.directed or s >= v:
                    for i in range(G.adj[s][v]):
                        dot += "  " + str(s) + link + str(v) + '\n'
    dot += '}'
    return dot

In [53]:
print(todot(graph_1_m))

digraph G {
  0 -> 1
  0 -> 2
  0 -> 6
  1 -> 3
  2 -> 6
  2 -> 8
  3 -> 6
  4 -> 3
  5 -> 2
  5 -> 6
  6 -> 3
  6 -> 4
  7 -> 5
  7 -> 6
  7 -> 8
}


### Cas de l'implémentation par listes d'adjacences

In [54]:
def todot(G):
    arrow = " -" + ("> " if G.directed else "- ")
    res = "digraph {\n" if G.directed else "graph {\n"
    for s in range(G.order):
        for v in G.adjlists[s]:
            if G.directed or v < s:
                res += "  {}{}{}\n".format(str(s), arrow, str(v))
    return res + "}"

In [55]:
print(todot(graph_1))

digraph {
  0 -> 1
  0 -> 6
  0 -> 2
  1 -> 3
  2 -> 6
  2 -> 8
  3 -> 6
  4 -> 3
  5 -> 2
  5 -> 6
  6 -> 3
  6 -> 4
  7 -> 6
  7 -> 5
  7 -> 8
}


In [56]:
print(todot(graph_2))

graph {
  1 -- 0
  2 -- 0
  3 -- 1
  5 -- 4
  6 -- 4
  7 -- 4
  7 -- 6
  7 -- 5
  8 -- 7
  8 -- 5
}


# Parcours

Sauf indication du contraire on suppose qu'on travaille avec des graphes simples (sans boucles ni arêtes/arcs multiples).

## Exercice 2.1 : parcours largeur

Il s'agit de parcourir un graphe en partant d'un sommet <tt>s</tt> puis tous les successeurs de celui-ci puis les successeurs de chacun des successeurs et ainsi de suite. Lors du parcours à partir de <tt>s</tt> on garde en mémoire le père de chaque sommet découvert. On choisi de le stocker dans une liste des pères; à l'indice <tt>i</tt> elle contient le père de <tt>i</tt>.

Pour ce parcours inspirez-vous du parcours profondeur sur les arbres.

Voici la fonction d'appel que vous aurez à utiliser :

In [57]:
from algopy import queue

### Avec l'implémentation matricielle

In [58]:
def bfs_m(G):
    p = [None]*G.order
    for s in range(G.order):
        if p[s] is None:
            p[s] = -1
            __bfs_m(G, s, p)

In [59]:
def __bfs_m(G, s, p):
    q = queue.Queue()
    q.enqueue(s)
    while not q.isempty():
        vertex = q.dequeue()
        print(vertex, end=" ")
        for v in range(G.order):
            if G.adj[vertex][v]:
                if p[v] == None:
                    q.enqueue(v)
                    p[v] = vertex

In [60]:
bfs_m(graph_2_m)

0 1 2 3 4 5 6 7 8 

### Avec l'implémentation par listes d'adjacences

In [61]:
def bfs(G):
    p = [None]*G.order
    for s in range(G.order):
        if p[s] is None:
            p[s] = -1
            __bfs(G, s, p)

In [62]:
def __bfs(G, s, p):
    q = queue.Queue()
    q.enqueue(s)
    while not q.isempty():
        vertex = q.dequeue()
        print(vertex, end=" ")
        for v in G.adjlists[vertex]:
                if p[v] == None:
                    q.enqueue(v)
                    p[v] = vertex

In [63]:
bfs(graph_2)

0 2 1 3 4 5 6 7 8 

## Exercice 2.2 : Parcours Profondeur

Le principe du parcours profondeur est de passer d'un sommet vers un successeur puis au successeur de ce dernier jusqu'à ne plus avoir de successeurs non visités, on reprend au premier sommet ayant un successeur non visité. C'est une généralisation du parcours profondeur pour les arbres.

### Avec l'implémentation par listes d'adjacences

#### On choisit de renvoyer la forêt couvrante

In [64]:
def __dfs(G, s, p):
    for v in G.adjlists[s]:
        if p[v] is None:
            print(v, end=" ")
            p[v] = s
            __dfs(G, v, p)
    
def dfs(G):
    p = [None]*G.order
    for s in range(G.order):
        if p[s] is None:
            p[s] = -1
            __dfs(G, s, p)
    return p        

In [65]:
dfs(graph_2)

2 1 3 5 7 6 8 

[-1, 0, 0, 1, -1, 4, 7, 5, 7]

#### On choisit de renvoyer les arcs retour

On sous-entend ici qu'on travail sur un graph non-orienté en entrée.

In [71]:
def __dfsback(G, s, p):
     for v in G.adjlists[s]:
        if p[v] is None:
            # print(v, end=" ")
            p[v] = s
            __dfsback(G, v, p)
        else:
            if v != p[s]:
                print(s, "->", v, "backward edge.")

def dfsback(G):
    p = [None]*G.order
    for s in range(G.order):
        if p[s] is None:
            p[s] = -1
            __dfsback(G, s, p)
    return p        

In [72]:
dfsback(graph_2)

7 -> 4 backward edge.
6 -> 4 backward edge.
8 -> 5 backward edge.
5 -> 8 backward edge.
4 -> 6 backward edge.
4 -> 7 backward edge.


[-1, 0, 0, 1, -1, 4, 7, 5, 7]

#### On choisit de travailler les arcs avants, retours ou croisés

On sous-entend dans ce cas qu'on travaille avec les graphes orientés. Sans quoi les seuls arcs qui ne sont pas dans la forêt sont des arcs retours.

In [70]:
def __depthfull(G, s, pref, suff, cpt):
    cpt += 1
    pref[s] = cpt
    for adj in G.adjLists[s]:
        if pref[adj] == 0:
            print (s, "->",  adj)
            cpt = __depthfull(G, adj, pref, suff, cpt)
        else:
            if pref[s] < pref[adj]:
                print (s, "->",  adj, "forward")
            else:
                if suff[adj] == 0:
                    print (s, "->",  adj, "back")
                else:
                    print (s, "->",  adj, "cross")
    cpt += 1
    suff[s] = cpt
    return cpt

def depthfull(G):
    pref = [0] * G.order
    suff = [0] * G.order 
    cpt = 0
    for s in range(G.order):
        if pref[s] == 0:
            cpt = __depthfull(G, s, pref, suff, cpt)
    return(pref, suff)

### Implémentation avec matrice d'adjacence

#### Cas des arcs retours uniquement

Retour sur les graphes non-orientés.

In [74]:
def __dfsback_m(G, s, p):
    for adj in range(G.order):
        if G.adj[s][adj]:
            if p[adj] == None:  # tree edge
                p[adj] = s      # vertices has to be marked here
                print(s, "->", adj)
                __dfsback_m(G, adj, p)
            else:
                if adj != p[s]:
                    print(s, '->', adj, "back edge")    #unless adj -> s is a back edge!
                
def dfsback_m(G):
    p = [None] * G.order
    for s in range(G.order):
        if p[s] == None:
            p[s] = -1
            __dfsback_m(G, s, p)
    return p

#### Cas des arcs avants, retours et croisés

A nouveau le cas des graphs orientés.

In [75]:
def __dephtfull_m(G, s, pref, suff, cpt):
    cpt += 1
    pref[s] = cpt
    for adj in range(G.order):
        if G.adj[s][adj]:
            if pref[adj] == 0:
                print(s, "->", adj)
                cpt = __dephtfull_m(G, s, pref, suff, cpt)
            else :
                if pref[s] < pref[adj]:
                    print(s, "->", adj, "is forward")
                else:
                    if suff[adj] == 0:
                        print(s, "->", adj, "is backward")
                    else:
                        print(s, "->", adj, "is cross")
    cpt += 1
    suff[s] = cpt
    return cpt


def depthfull_m(G):
    pref = [0] * G.order
    suff = [0] * G.order 
    cpt = 0
    for s in range(G.order):
        if pref[s] == 0:
            cpt = __depthfull_m(G, s, pref, suff, cpt)
    return(pref, suff)

# Quelques applications

## Exercice 3.1 : recherche de trajet

On a deux manières de trouver un chemin entre un sommet <tt>s</tt> et un autre <tt>v</tt> dans un graphe <tt>G</tt> :

- En faisant un parcours profondeur au départ de <tt>s</tt> puis, une fois <tt>v</tt> trouvé renvoyer la liste des antécédants de <tt>v</tt> dans l'arbre couvrant. 
- En faisant un parcours largeur au départ de <tt>s</tt> puis, une fois <tt>v</tt> trouvé renvoyer la liste des antécédants de <tt>v</tt> dans l'arbre couvrant.

Bien entendu, si <tt>v</tt> n'est pas atteint lors des parcours précédants il n'y a pas de chemin de <tt>s</tt> à <tt>v</tt>. L'intérêt du parcours largeur vient du fait qu'on obtient un plus court chemin de <tt>s</tt> à <tt>v</tt>. Dans le cas d'un parcours profondeur le chemin n'est cependant pas un plus court chemin.

### Parcours profondeur

In [76]:
def __pathDFS(G, s, v, M, path):
    for adj in G.adjLists[s]:
        M[adj] = True
        if adj == v or __pathDFS(G, adj, v, M, path):
            path.insert(0, adj)
            return(True)
    return(False)

def pathDFS(G, s, v):
    M = [False]*G.order
    path = [] 
    if __pathDFS(G, s, v, M, path):
        path.insert(0, s)
    return(path)

### Parcours largeur

In [77]:
def __pathBFS(G, src, dst, p):
    q = queue.Queue()
    q = queue.enqueue(src, q)
    p[src] = -1
    while not queue.isEmpty(q):
        s = queue.dequeue(q)
        for adj in G.adjLists[s]:
            if p[adj] == None:       
                p[adj] = s
                if adj == dst:
                    return True
                q = queue.enqueue(adj, q)
            
    return False

def pathBFS(G, src, dst):
    p = [None] * G.order
    L = []
    if __pathBFS(G, src, dst, p):
        while dst != -1:
            L.insert(0, dst)
            dst = p[dst]
    return L

## Exercice 3.2 : distance au départ

Le parcours largeur calcule les distances entre le point de départ de la recherche et n'importe quel deux sommets de l'arbre. Il suffit à partir de là de repérer quand on se retrouve à distance entre `d_min` et `d_max`.

In [81]:
def __distances(G, s, M, dmin, dmax):    
    q = queue.Queue()
    f = queue.Queue()
    q.enqueue(s)
    M[s] = True
    depth = 0
    while not q.isempty():
        s = q.dequeue()
        if depth >= dmin:
            print(s, end=' ')
        if depth < dmax:
            for adj in range(G.order):
                if G.adj[s][adj]:
                    if not M[adj]:
                        f.enqueue(adj)
                        M[adj] = True
        if q.isempty():
            depth += 1
            if depth > dmin:
                print()
            (q, f) = (f, q)
            
    
def distances(G, src, dmin, dmax):
    M = [False] * G.order
    __distances2(G, src, M, dmin, dmax)

## Exercice 3.3 : I want to be tree

Il s'agit de repérer les arcs retours. C'est précisément ceux-ci qui permettent de repérer les cylces d'un graphe.

In [35]:
def __istree(G, s, p):
    for v in G.adjlists[s]:
        if p[v] is None:
            p[v] = s
            if not __istree(G, v, p):
                return False
        else:
            if v != p[s]:
                return False
    return True
    
def istree(G):
    p = [None]*G.order
    p[0] = -1
    if __istree(G, 0, p):
        j = 0
        while j < G.order and p[j] != None:
            j += 1
        return j == G.order
    else:
        return False

## Exercice 3.4 : diamètre

Pour calculer le diamètre on calcule la plus grande distance de la racine à une des feuilles de l'arbre. À partir de cette feuille on recalcule la plus grande distance de celle-ci à l'un des sommets de l'arbre. On aura dans un tel cas "déplié" la plus grande branche de l'arbre.

In [79]:
def __diam(G, s, m):
    q = queue.Queue()
    f = queue.Queue()
    i = -1
    q.enqueue(s)
    while not q.isempty():
        s = q.dequeue()
        for adj in G.adjlists[s]:
            if not m[adj]:
                m[adj] = True
                f.enqueue(adj)
        if q.isempty():
            q, f = f, q
            i = i + 1
    return  s, i 
    
    
def diam(G):
    if not G.order:
        raise Exception("Graph is empty. Diameter is not well defined.")
        
    m = [False]*G.order
    m[0] = True
    s, _ = __diam(G, 0, m)
    
    m = [False]*G.order
    m[s] = True
    _ , d = __diam(G, s, m)
    
    return d

## Exercice 3.5 : un problème d'ordonnancement


Les problèmes d'ordonnancement qu'on est en mesure de traiter sont des problèmes de tris topologiques. Il s'agit d'ordonner les sommets d'un graphe orienté de manière à ce que les extrémités d'un arc <tt>(dst, src)</tt> soient toujours ordonnés par ordre croissant. Bien évidemment celà n'est pas possible dans le cas de l'existence d'un circuit. 

### Principe 

Pour effectuer un tel tri il suffit de se souvenir du fait suivant : lors d'un parcours profondeur d'un graphe le passage suffixe sur un sommet <tt>s</tt> signifie qu'on a déjà visité tous les successeurs de <tt>s</tt>. À ce stade aucun des sommets qui apparaitront dans la suite du parcours n'ont de chemin vers <tt>s</tt>. Les sommets qui apparaissent à la suite dans ce parcours seront ajoutés après <tt>s</tt> dans l'ordre, les autres avant.

In [78]:
def __toporder(G, s, M, L):
    M[s] = True
    for adj in G.adjLists[s]:
        if not M[adj]:
            __toporder(G, adj, M, L)
    L.insert(0, s)

def toporder(G):
    M = [False] * G.order
    L = []
    for s in range(G.order):
        if not M[s]:
            __toporder(G, s, M, L)
    return L